<a href="https://colab.research.google.com/github/orlandoalexander/Moda-Personal-Assistant/blob/models/Nigel_Attribute_ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Input
import pandas as pd
import numpy as np


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !pip install --upgrade pip


In [4]:
!pip install git+https://github.com/orlandoalexander/Moda-Personal-Assistant.git@preproc_package


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/orlandoalexander/Moda-Personal-Assistant.git (to revision preproc_package) to /tmp/pip-req-build-leo4ftkm
  Running command git clone -q https://github.com/orlandoalexander/Moda-Personal-Assistant.git /tmp/pip-req-build-leo4ftkm
  Running command git checkout -b preproc_package --track origin/preproc_package
  Switched to a new branch 'preproc_package'
  Branch 'preproc_package' set up to track remote branch 'preproc_package' from 'origin'.
     |████████████████████████████████| 11.7 MB 7.7 MB/s 
     |████████████████████████████████| 17.1 MB 86.8 MB/s 
     |████████████████████████████████| 11.3 MB 71.9 MB/s 
     |████████████████████████████████| 4.5 MB 82.4 MB/s 
     |████████████████████████████████| 61 kB 131 kB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 31.2 MB 361 kB/s 
     |█████████████████

In [5]:
# Uncomment as needed
!unzip '/content/drive/MyDrive/attribute_img/img.zip'

Streaming output truncated to the last 5000 lines.
  inflating: img/Woven_Dolphin_Shorts/img_00000027.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000028.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000029.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000030.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000031.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000032.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000033.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000034.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000035.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000036.jpg  
  inflating: img/Woven_Dolphin_Shorts/img_00000037.jpg  
   creating: img/Woven_Drawstring_Joggers/
  inflating: img/Woven_Drawstring_Joggers/img_00000001.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000002.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000003.jpg  
  inflating: img/Woven_Drawstring_Joggers/img_00000004.jpg  
  inflating: img/Woven_Drawstring_J

In [6]:
from preproc.preprocess import AttributePreproc


In [7]:
# input size for resnet, efficientnet and mobilenet is (224,224). For inceptionv3 use 299,299
prep = AttributePreproc('img', (224,224), 'neckline', 0.2) 

In [8]:
X_train, X_test, y_train, y_test, names  = prep.run()

Augmenting attribute 'crew_neckline'...
Augmenting attribute 'v_neckline'...
Augmenting attribute 'square_neckline'...
Augmenting attribute 'no_neckline'...
Done!


In [9]:
## New Class code




In [10]:
## Orginal Class code

from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Lambda, BatchNormalization, Dropout
from keras import Sequential, Model
from keras.metrics import Precision, Recall
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from keras.applications import InceptionV3
from keras.applications import inception_v3
from keras.applications import ResNet50
from keras.applications import MobileNetV2
from keras.applications import MobileNet
from keras.applications import mobilenet_v2
from keras.applications import EfficientNetB0
from keras.applications import efficientnet
from keras.applications import mobilenet
from keras.applications import resnet
from keras.applications import inception_v3

from sklearn.model_selection import train_test_split

# attribute should be one of the following strings:
# 'design', 'sleeves', 'length', 'neckline', 'fabric', 'fit'

# model should be one of the following strings:
# 'inception', 'resnet', 'mobilenet', 'efficientnet'

class AttrModel(Model):
    def __init__(self, attribute, epochs, model_name, in_shape, batch_size, callbacks, final_layer_neurons,
                 pretrain_lr, finetune_lr,
                 X_train, X_test,
                 y_train, y_test, **kwargs):
        super().__init__()
        self.attribute = attribute
        self.model_name = model_name.lower()
        self.in_shape = in_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.pretrain_lr = pretrain_lr
        self.finetune_lr = finetune_lr
        self.callbacks = callbacks
        self.final_layer_neurons = final_layer_neurons
        self.kwargs = kwargs
        self.X_pretrain, self.X_finetune, self.y_pretrain, self.y_finetune = train_test_split(
            X_train, y_train, test_size=0.5
        )
        self.X_test = X_test
        self.y_test = y_test
        self.cat_nums = {
            'design': 7,
            'sleeves': 3,
            'length': 1,      # Binary category = 1
            'neckline': 4,
            'fabric': 6,
            'fit': 3
        }

        del X_train
        del y_train

        self.cat_num = self.cat_nums[self.attribute]
        self.activation = 'sigmoid' if self.attribute == 'length' else 'softmax'
        self.loss = 'binary_crossentropy' if self.attribute == 'length' else 'categorical_crossentropy'
        self.model = self.instantiate_model()  # calling the function below

    def instantiate_model(self):
        input = Input(self.in_shape)
        #preprocess = Lambda(mobilenet.preprocess_input)

        if self.model_name == 'inception':              # calling the chosen pretrained model
            base_model = InceptionV3(include_top=False, weights='imagenet',
                                    classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = inception_v3.preprocess_input
        elif self.model_name == 'resnet':
            base_model = ResNet50(include_top=False, weights='imagenet',
                                  classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = resnet.preprocess_input
        elif self.model_name == 'mobilenet':
            base_model = MobileNetV2(include_top=False, weights='imagenet',
                                     classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = mobilenet.preprocess_input
        elif self.model_name == 'efficientnet':
            base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                        classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = efficientnet.preprocess_input
        else:
            print('''No model found. Please pass one of the following:
                  inception, resnet, mobilenet, efficientnet''')

        base_model.trainable = False    # freeze layers
        outputs = base_model(input)
        outputs = BatchNormalization(name='BatchNormalization')(outputs)
        outputs = GlobalAveragePooling2D()(outputs)
        outputs = Dropout(0.5)(outputs)
        outputs = Dense(units=self.final_layer_neurons, activation='relu')(outputs)
        outputs = LeakyReLU(alpha=0.1)(outputs)
        outputs = Dropout(0.25)(outputs)
        outputs = Dense(units=self.cat_num, activation=self.activation)(outputs)
        self.model = Model(inputs = [input], outputs = [outputs])
        self.model.save('')
        print(self.loss) # check loss func
        self.model.compile(loss=self.loss, optimizer=Adam(learning_rate=self.pretrain_lr),
                      metrics=['accuracy', Precision(), Recall()])
        return self.model

    def train(self):
        datagen = ImageDataGenerator(preprocessing_function=self.preprocess_input)

        self.pretrain_generator = datagen.flow( 
        self.X_pretrain, self.y_pretrain,                                
        batch_size=16)

        self.finetune_generator = datagen.flow( 
        self.X_finetune, self.y_finetune,                                
        batch_size=16)

        self.test_generator = datagen.flow( 
        self.X_test, self.y_test,                                
        batch_size=16)

        # Train the model on the images and labels
        self.model.history = self.model.fit(self.pretrain_generator,
        epochs=self.epochs, batch_size=self.batch_size, verbose=1, 
        callbacks=self.callbacks)

        return self.model.history

    def finetune(self,
                 metrics=['accuracy', Precision(), Recall()]):
      
        self.model.trainable = True     # unfreeze layers, then compile to save changes
        self.model.compile(
            optimizer=Adam(self.finetune_lr),                  # Very low learning rate
            loss=self.loss,
            metrics=metrics)
        self.model.fit(self.finetune_generator,
                       epochs=self.epochs, batch_size=self.batch_size,
                       callbacks=self.callbacks)
        return self.model, self.model.history

    def evaluate(self):
        return self.model.evaluate(self.test_generator, verbose=1)

    def predict(self, X):
        return self.model.predict(self.preprocess_input(X))


    def save(self, filepath):
        return tf.keras.models.save_model(self.model,
                                   filepath=filepath)
        
    def summary(self):
        return self.model.summary()


In [11]:
efficientnet_neckline = AttrModel(attribute='neckline',
                      model_name='efficientnet',
                      in_shape=(224,224,3),
                      batch_size= 32,
                      epochs=25,
                      pretrain_lr = .0001,
                      finetune_lr = .00001,
                      callbacks = EarlyStopping(patience=3, restore_best_weights=True, monitor='accuracy'),
                      final_layer_neurons=100,
                      X_train=X_train,
                      X_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

16705208/16705208 [==============================] - 1s 0us/step


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7f9fc4371d30>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7f9fc4371d30>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
categorical_crossentropy


In [12]:
efficientnet_neckline.train()


Epoch 1/25


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
405/405 [==============================] - 29s 37ms/step - loss: 1.2230 - accuracy: 0.4679 - precision_1: 0.6101 - recall_1: 0.2101
Epoch 2/25
405/405 [==============================] - 14s 35ms/step - loss: 0.9938 - accuracy: 0.6016 - precision_1: 0.7142 - recall_1: 0.4166
Epoch 3/25
405/405 [==============================] - 14s 35ms/step - loss: 0.8922 - accuracy: 0.6432 - precision_1: 0.7379 - recall_1: 0.5009
Epoch 4/25
405/405 [==============================] - 14s 36ms/step - loss: 0.8522 - accuracy: 0.6572 - precision_1: 0.7447 - recall_1: 0.5263
Epoch 5/25
405/405 [==============================] - 15s 36ms/step - loss: 0.8114 - accuracy: 0

In [13]:
efficientnet_neckline.finetune()

Epoch 1/25


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
405/405 [==============================] - 77s 167ms/step - loss: 1.1933 - accuracy: 0.4912 - precision: 0.5561 - recall: 0.3460
Epoch 2/25
405/405 [==============================] - 67s 165ms/step - loss: 0.9329 - accuracy: 0.5887 - precision: 0.6688 - recall: 0.4774
Epoch 3/25
405/405 [==============================] - 67s 165ms/step - loss: 0.8393 - accuracy: 0.6403 - precision: 0.7105 - recall: 0.5352
Epoch 4/25
405/405 [==============================] - 67s 165ms/step - loss: 0.7796 - accuracy: 0.6701 - precision: 0.7390 - recall: 0.5743
Epoch 5/25
405/405 [==============================] - 67s 165ms/step - loss: 0.7199 - accuracy: 0.6974 - pre

(<keras.engine.functional.Functional at 0x7f9fc541b7c0>,
 <keras.callbacks.History at 0x7f9f44d830d0>)

In [14]:
efficientnet_neckline.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 BatchNormalization (BatchNo  (None, 7, 7, 1280)       5120      
 rmalization)                                                    
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 100)               128100

In [15]:
efficientnet_neckline.evaluate()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
203/203 [==============================] - 10s 37ms/step - loss: 0.6829 - accuracy: 0.7510 - precision: 0.7687 - recall: 0.7300


[0.6828515529632568, 0.7510040402412415, 0.7687052488327026, 0.729996919631958]

In [22]:
efficientnet_neckline.save('/content/drive/MyDrive/Moda/saved_models/efficientnet_neckline_v1')

Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7f9fc5d89af0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x7f9fc5d89af0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('tape_grad_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
efficientnet_neckline_v1_metrics = {
    'loss': 0.6829,
    'accuracy': 0.7510,
    'precision': 0.7687,
    'recall': 0.7300
    }

In [ ]:
import json

In [ ]:
# Writing to sample.json
with open('/content/drive/MyDrive/Moda/saved_models/efficientnet_neckline_v1/eval_metrics.txt', 'w') as outfile:
    outfile.write(json.dumps(efficientnet_neckline_v1_metrics))

In [ ]:
del prep

In [ ]:
del X_train, y_train, X_test, y_test